# Notebook Overview

## Inputs
The input variables for all trials in this notebook are the following:
```py
include_fields = ['day_of_week','hours_l1','hours_l2','hours_l3','hours_l4',
                  'hours_l5','hours_l6','hours_l7','hours_l8','hours_l14','avg_employees',
                  'perc_hours_today_before', 'perc_hours_yesterday_before', 'perc_hours_tomorrow_before']
```

## Models and Hyperparameters
The following hyperparameters are explored using a grid search. 
```py
Num_Layers = [2,4,6,8]
Num_Units = [8,16,24,32]
Dropout_Rate = [0.25,0.5,0.75]
```
Each combination of num_layers, num_units, and dropout_rate is trained on the train data and validated on the crossvalidation data. The outputted numbers in the cells below correspond to the crossvalidation loss and metric scores. 

For a given permuation {l,u,r} of {num_layers,num_units,dropout_rate}, a model contains (l-1) layers of width u, each with dropout rate r applied to its output, connected to a final layer that outputs a prediction. 

In [ ]:
import pandas as pd
import time
import tensorflow as tf


include_fields = ['hours','day_of_week','hours_l1','hours_l2','hours_l3','hours_l4',
                  'hours_l5','hours_l6','hours_l7','hours_l8','hours_l14','avg_employees',
                  'perc_hours_today_before', 'perc_hours_yesterday_before', 'perc_hours_tomorrow_before']

startTime = time.time()
train = pd.read_csv("/export/storage_adgandhi/PBJhours_ML/Data/Intermediate/train_test_validation/training_set.csv",usecols=include_fields).dropna()
val = pd.read_csv("/export/storage_adgandhi/PBJhours_ML/Data/Intermediate/train_test_validation/crossvalidation_set.csv",usecols=include_fields).dropna()
print(f"Loaded Train and Validation sets. Time taken: {time.time()-startTime}")
print(train.head)

In [ ]:
train_inputs, train_labels = train.drop(['hours'], axis=1), train.filter(['hours'])
val_inputs, val_labels = val.drop(['hours'], axis=1), val.filter(['hours'])

In [ ]:
#appends one hot expansion of selected labels to end of dataframe (axis 1)
def expand_one_hot(labels,dataset):
    outList = []
    for label in labels:  
        col = dataset[label]
        ###Generate a dict for all unique values (Don't waste space encoding non important job id's)
        map = {}
        index = 0
        for element in col.unique():
            map[element] = index
            index += 1
        col = col.map(map)
        tensor = tf.one_hot(col,len(col.unique()))
        outList.append(tensor)
        dataset = dataset.drop(columns=[label])
    
    outList.append(dataset)
    output = tf.concat(outList,1)
    return output

train_inputs = expand_one_hot(['day_of_week'],train_inputs)
val_inputs = expand_one_hot(['day_of_week'],val_inputs)

print(train_inputs.shape)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
BUFFER_SIZE = 10000
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
BATCH_SIZE_PER_REPLICA = 512
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
trainSet = tf.data.Dataset.from_tensor_slices((train_inputs,train_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
valSet = tf.data.Dataset.from_tensor_slices((val_inputs,val_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

print(trainSet)

In [ ]:
def train_test_model(hparams):
    with strategy.scope():
        model = tf.keras.models.Sequential()

        #Build model to depth specificed in hparams[HP_NUM_LAYERS]
        for i in range(hparams[HP_NUM_LAYERS]):
            if i+1 == hparams[HP_NUM_LAYERS]:
                model.add(tf.keras.layers.Dense(1))
            elif i == 0:
                model.add(tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu))
            else:
                model.add(tf.keras.layers.Dropout(hparams[HP_DROPOUT]))
                model.add(tf.keras.layers.Dense(hparams[HP_NUM_UNITS], activation=tf.nn.relu))           
    
        model.compile(
            loss=tf.keras.losses.MeanSquaredError(),
            optimizer=tf.keras.optimizers.Adam(),
            metrics=[tf.keras.metrics.MeanAbsoluteError()]
        )
    
    callback = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', min_delta=0.01, patience=1, verbose=0,
                    mode='auto', baseline=None, restore_best_weights=False
                )
    
    startTime = time.time()
    model.fit(trainSet, epochs=10, verbose=0, validation_data=valSet, callbacks=[callback]) 
    _, accuracy = model.evaluate(valSet)
    return accuracy, time.time()-startTime

In [ ]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy, time = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        print(f"MAE: {accuracy}    Time Taken: {time} seconds")

In [ ]:
from tensorboard.plugins.hparams import api as hp

HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([8,16,24,32]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.25, 0.5, 0.75]))
HP_NUM_LAYERS = hp.HParam('num_layers', hp.Discrete([2,4,6,8]))

METRIC_ACCURACY = 'Mean Squared Error'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_NUM_UNITS, HP_DROPOUT, HP_NUM_LAYERS],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Mean Squared Error')],
    )

In [ ]:
session_num = 0
for num_layers in HP_NUM_LAYERS.domain.values:
    for dropout_rate in HP_DROPOUT.domain.values:
        for num_units in HP_NUM_UNITS.domain.values:
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_NUM_LAYERS: num_layers,
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-0
{'num_units': 8, 'dropout': 0.25, 'num_layers': 2}
536944/536944 [==============================] - 587s 1ms/step - loss: 13.7086 - mean_absolute_error: 3.1174
MAE: 3.1173713207244873    Time Taken: 4153.185551643372 seconds
--- Starting trial: run-1
{'num_units': 16, 'dropout': 0.25, 'num_layers': 2}
536944/536944 [==============================] - 579s 1ms/step - loss: 13.6644 - mean_absolute_error: 3.0769
MAE: 3.076856851577759    Time Taken: 4123.465849399567 seconds
--- Starting trial: run-2
{'num_units': 24, 'dropout': 0.25, 'num_layers': 2}
536944/536944 [==============================] - 606s 1ms/step - loss: 13.5299 - mean_absolute_error: 3.0546
MAE: 3.0546138286590576    Time Taken: 6057.240978717804 seconds
--- Starting trial: run-3
{'num_units': 32, 'dropout': 0.25, 'num_layers': 2}


In [ ]:
session_num = 3
for num_layers in HP_NUM_LAYERS.domain.values:
    for dropout_rate in HP_DROPOUT.domain.values:
        for num_units in HP_NUM_UNITS.domain.values:
            if int(num_layers) == 2 and float(dropout_rate) == 0.25 and int(num_units) != 32:
                continue
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_NUM_LAYERS: num_layers,
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-3
{'num_units': 32, 'dropout': 0.25, 'num_layers': 2}
536944/536944 [==============================] - 606s 1ms/step - loss: 13.4821 - mean_absolute_error: 3.0569
MAE: 3.056939125061035    Time Taken: 6314.490927457809 seconds
--- Starting trial: run-4
{'num_units': 8, 'dropout': 0.5, 'num_layers': 2}
536944/536944 [==============================] - 611s 1ms/step - loss: 14.2999 - mean_absolute_error: 3.2726
MAE: 3.2726030349731445    Time Taken: 4357.095632314682 seconds
--- Starting trial: run-5
{'num_units': 16, 'dropout': 0.5, 'num_layers': 2}
536944/536944 [==============================] - 611s 1ms/step - loss: 13.7397 - mean_absolute_error: 3.1449
MAE: 3.1449296474456787    Time Taken: 6241.574224233627 seconds
--- Starting trial: run-6
{'num_units': 24, 'dropout': 0.5, 'num_layers': 2}
536944/536944 [==============================] - 612s 1ms/step - loss: 13.7729 - mean_absolute_error: 3.1238
MAE: 3.1237761974334717    Time Taken: 4366.5706615448 seconds

In [ ]:
session_num = 13
for num_layers in HP_NUM_LAYERS.domain.values:
    for dropout_rate in HP_DROPOUT.domain.values:
        for num_units in HP_NUM_UNITS.domain.values:
            if int(num_layers) == 2:
                continue
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_NUM_LAYERS: num_layers,
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-13
{'num_units': 8, 'dropout': 0.25, 'num_layers': 4}
536944/536944 [==============================] - 627s 1ms/step - loss: 14.8297 - mean_absolute_error: 3.3363
MAE: 3.336268186569214    Time Taken: 4778.473185777664 seconds
--- Starting trial: run-14
{'num_units': 16, 'dropout': 0.25, 'num_layers': 4}
536944/536944 [==============================] - 626s 1ms/step - loss: 13.9795 - mean_absolute_error: 3.1706
MAE: 3.170628070831299    Time Taken: 4801.904301166534 seconds
--- Starting trial: run-15
{'num_units': 24, 'dropout': 0.25, 'num_layers': 4}
536944/536944 [==============================] - 619s 1ms/step - loss: 13.8331 - mean_absolute_error: 3.1596
MAE: 3.15960955619812    Time Taken: 4869.2269151210785 seconds
--- Starting trial: run-16
{'num_units': 32, 'dropout': 0.25, 'num_layers': 4}
536944/536944 [==============================] - 640s 1ms/step - loss: 13.3729 - mean_absolute_error: 3.0682
MAE: 3.0681569576263428    Time Taken: 7339.327345371246 

In [ ]:
session_num = 18
for num_layers in HP_NUM_LAYERS.domain.values:
    for dropout_rate in HP_DROPOUT.domain.values:
        for num_units in HP_NUM_UNITS.domain.values:
            if int(num_layers) == 2:
                continue
            if int(num_layers) == 4 and float(dropout_rate) == 0.25:
                continue
            if int(num_layers) == 4 and float(dropout_rate) == 0.50 and int(num_units)==8:
                continue
            hparams = {
                HP_NUM_UNITS: num_units,
                HP_DROPOUT: dropout_rate,
                HP_NUM_LAYERS: num_layers,
            }
            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-18
{'num_units': 16, 'dropout': 0.5, 'num_layers': 4}
268472/268472 [==============================] - 307s 1ms/step - loss: 14.0474 - mean_absolute_error: 3.1686
MAE: 3.168642520904541    Time Taken: 2478.577969312668 seconds
--- Starting trial: run-19
{'num_units': 24, 'dropout': 0.5, 'num_layers': 4}
268472/268472 [==============================] - 314s 1ms/step - loss: 14.1113 - mean_absolute_error: 3.1752
MAE: 3.1751532554626465    Time Taken: 2455.9027378559113 seconds
--- Starting trial: run-20
{'num_units': 32, 'dropout': 0.5, 'num_layers': 4}
268472/268472 [==============================] - 321s 1ms/step - loss: 14.0940 - mean_absolute_error: 3.1547
MAE: 3.154683828353882    Time Taken: 3624.537680387497 seconds
--- Starting trial: run-21
{'num_units': 8, 'dropout': 0.75, 'num_layers': 4}
268472/268472 [==============================] - 329s 1ms/step - loss: 18.1709 - mean_absolute_error: 3.8770
MAE: 3.8770172595977783    Time Taken: 4693.5349752902985 